# Chatbot Service API Demo

This notebook demonstrates how to interact with the Chatbot Service API:
- Sending chat messages
- Retrieving chat sessions
- Using debug mode to see RAG chunks
- Parsing responses


In [ ]:
import httpx
import json
from typing import Dict, Any
import asyncio

# Configuration
BASE_URL = "http://localhost:8003"  # Adjust if your service runs on different port


## 1. Send a Chat Message


In [ ]:
async def send_chat_message(
    session_id: str,
    user_id: str,
    text: str,
    current_course_id: str | None = None,
    debug: bool = False
) -> Dict[str, Any]:
    """Send a chat message to the chatbot API."""
    async with httpx.AsyncClient() as client:
        response = await client.post(
            f"{BASE_URL}/api/v1/chat/messages",
            json={
                "session_id": session_id,
                "user_id": user_id,
                "text": text,
                "current_course_id": current_course_id,
                "debug": debug
            }
        )
        response.raise_for_status()
        return response.json()

# Example: Ask a question about a course
result = await send_chat_message(
    session_id="demo-session-1",
    user_id="user123",
    text="What is the difference between a list and a tuple in Python?",
    current_course_id="course_python_basic",
    debug=False
)

print("Reply:", result["reply"])


## 2. Debug Mode: See Retrieved Chunks


In [ ]:
# Enable debug mode to see which chunks were used for RAG
result = await send_chat_message(
    session_id="demo-session-2",
    user_id="user123",
    text="How do I define a list in Python?",
    current_course_id="course_python_basic",
    debug=True
)

print("Reply:", result["reply"])
print("\nDebug Info:")
if result.get("debug"):
    for i, chunk in enumerate(result["debug"]["chunks"], 1):
        print(f"\nChunk {i}:")
        print(f"  Course ID: {chunk['course_id']}")
        print(f"  Score: {chunk.get('score', 'N/A')}")
        print(f"  Preview: {chunk['text_preview'][:100]}...")


## 3. Retrieve Chat Sessions


In [ ]:
async def get_sessions(user_id: str) -> list:
    """Get all chat sessions for a user."""
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{BASE_URL}/api/v1/chat/sessions",
            params={"user_id": user_id}
        )
        response.raise_for_status()
        return response.json()

sessions = await get_sessions("user123")
print(f"Found {len(sessions)} sessions:")
for session in sessions:
    print(f"  - {session['id']}: {session.get('last_intent', 'N/A')}")


## 4. Get Session Details with Messages


In [ ]:
async def get_session_detail(session_id: str) -> Dict[str, Any]:
    """Get session details including all messages."""
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{BASE_URL}/api/v1/chat/sessions/{session_id}"
        )
        response.raise_for_status()
        return response.json()

session_detail = await get_session_detail("demo-session-1")
print(f"Session: {session_detail['session']['id']}")
print(f"Messages: {len(session_detail['messages'])}")
print("\nConversation:")
for msg in session_detail["messages"]:
    print(f"{msg['sender']}: {msg['text']}")
